# Stereo camera calibration
Intrinsic and Extrinsic parameters of single calibrated images

In [1]:
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
BOARD = (7,7)
squareSize = 0.125 #size of single chessboard square in mm
image_size = None

In [3]:
wp = np.zeros((1,BOARD[0]*BOARD[1],3), dtype=np.float32)
wp[0,:,:2] = np.mgrid[0:BOARD[0], 0:BOARD[1]].T.reshape(-1, 2)
wp = wp * squareSize

In [4]:
 # Arrays to store object points and image points from all the images.
wpoints = []  # 3d point in real world space
left_imgpoints = []  # 2d points in image plane.
right_imgpoints = []  # 2d points in image plane.

In [6]:
left_images = glob.glob('left/*.png')
right_images = glob.glob('right/*.png')

if len(left_images) != len(right_images):
    raise ValueError('Must have same number of files in both directories')
    print("Left images count: ", len(left_images))
    print("Right images count: ", len(right_images))
    raise

left_images.sort()
right_images.sort()
image_pairs = zip(left_images, right_images)

valid = 0
false = 0
for left,right in image_pairs:
    # Images
    imgL = cv2.imread(left,0) # left (monochrome)
    imgR = cv2.imread(right,0) # right (monochrome)
    
    # Find chessboard corners
    ret_left, corners_left = cv2.findChessboardCorners(imgL, BOARD,
                                                       flags=cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
                                                       
    ret_right, corners_right = cv2.findChessboardCorners(imgR, BOARD,
                                                       flags=cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    
    # If found, add object points and image points(refinied)
    if ret_left and ret_right:
        # Object points
        wpoints.append(wp)
        # Left points
        refined_corners_left = cv2.cornerSubPix(imgL, corners_left, (11,11),(-1,-1), criteria) 
        left_imgpoints.append(refined_corners_left) 
        imageL = cv2.drawChessboardCorners(imgL, BOARD, refined_corners_left, ret_left)
        # Right points
        refined_corners_right = cv2.cornerSubPix(imgR, corners_right, (11,11),(-1,-1), criteria) 
        right_imgpoints.append(refined_corners_left) 
        valid += 1 
        imageR = cv2.drawChessboardCorners(imgR, BOARD, refined_corners_right, ret_right)
    else:
        false += 1
        # points were not found
        #print("Chessboard wasn't detected. Image pair: ", left, " and ", right)
        #continue
    
    #cv2.imshow('Left Points', imageL)
    #cv2.imshow('Right Points', imageR)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
print("Corner detection Summary\nFrom the {} samples\n\
Accepted Pairs: {}\nRejected Pairs: {}".format(len(left_images),valid,false))

Corner detection Summary
From the 80 samples
Accepted Pairs: 22
Rejected Pairs: 58


In [7]:
# Calibrate cameras
print("Starting Calibration...")
h, w = imgL.shape
ret_l, K1, D1, rvec_l, tvec_l = cv2.calibrateCamera(wpoints, left_imgpoints, (w, h) , None, None)
n_K1, roi_l = cv2.getOptimalNewCameraMatrix(K1, D1, (w, h), 1, (w, h))

ret_r, K2, D2, rvec_r, tvec_r = cv2.calibrateCamera(wpoints, right_imgpoints,(w, h) , None, None)
n_K2, roi_r = cv2.getOptimalNewCameraMatrix(K2, D2, (w, h), 1, (w, h))

# Stereo calibration
flag = 0
flag = cv2.CALIB_SAME_FOCAL_LENGTH | cv2.CALIB_FIX_INTRINSIC
ret, intr_l, dist_l, intr_r, dist_r, R, T, E, F = cv2.stereoCalibrate(wpoints, left_imgpoints, right_imgpoints, n_K1, 
                                                                      D1, n_K2, D2, (w, h),flags=flag,criteria=criteria)
print("Calibration complete!")
# Display results
print("\nStereo Calibration Parameters\n")
print('rms: {}\n\nIntricsic matrix:\n{}\n\nDistortion coefficients:\n{}\n\nRotation vector:\n{}\n\
              \nTranslation vector:\n{}'.format(ret, intr_l, dist_l.ravel(), R, T),end="\n")

Starting Calibration...
Calibration complete!

Stereo Calibration Parameters

rms: 0.9531273238871255

Intricsic matrix:
[[2.39280981e+03 0.00000000e+00 1.01760868e+03]
 [0.00000000e+00 2.39226392e+03 1.02638707e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion coefficients:
[-7.92927220e-03  1.79030072e-01  4.92755444e-04 -4.74009098e-04
 -1.05186700e+00]

Rotation vector:
[[ 1.00000000e+00 -3.37889648e-14 -1.40571576e-14]
 [ 3.37889648e-14  1.00000000e+00  5.84283209e-14]
 [ 1.40571576e-14 -5.84283209e-14  1.00000000e+00]]
              
Translation vector:
[[ 4.06789822e-14]
 [-1.35485377e-13]
 [ 4.34541057e-14]]


In [7]:
# Rectify images
R1, R2, P1, P2, Q, J1, J2 = cv2.stereoRectify(intr_l, dist_l, intr_r, dist_r, (w, h), R, T, alpha=0.9)
leftMapX, leftMapY = cv2.initUndistortRectifyMap(intr_l, dist_l, R1, P1, (w, h), cv2.CV_32FC1)
left_rectified = cv2.remap(imageL, leftMapX, leftMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
rightMapX, rightMapY = cv2.initUndistortRectifyMap(intr_r, dist_r, R2, P2, (w, h), cv2.CV_32FC1)
right_rectified = cv2.remap(imageR, rightMapX, rightMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)

13